In [ ]:
from tqdm.notebook import tqdm
import os
import shutil
from pathlib import Path

import pandas as pd
import supervision as sv
import torch
from sklearn.model_selection import train_test_split

In [ ]:



############################################################################################
############# Формируем датасет из каталога Победоносцев (разметка сегментами) #############
############################################################################################


image_dir = "../../data/raw/Распознавание текстов/Победоносцев/images"
# annotation_file = "../../data/raw/Распознавание текстов/Победоносцев/project-14-at-2024-03-18-16-02-b43f1e84.json"
annotation_file = '../../data/raw/Распознавание текстов/Победоносцев/project-14-at-2024-03-27-22-12-07be0e2d.json'
segment_annotations = pd.read_json(annotation_file)

segment_images = []

# Перебор всех файлов изображений в image_dir
for root, dirs, files in os.walk(image_dir):
    for file in files:
        image_path = os.path.join(root, file)
        segment_images.append(image_path)
        
train_images, test_images = train_test_split(segment_images, test_size=0.2, random_state=42)
train_images, valid_images = train_test_split(train_images, test_size=0.2, random_state=42)

In [ ]:
import os
import random
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split


# Пути для сохранения разделенных выборок
images_text_segmenter_train_dir = "../../data/processed/4 Segmenter test/text_segmenter/train/images"
labels_text_segmenter_train_dir = "../../data/processed/4 Segmenter test/text_segmenter/train/labels"

images_text_segmenter_valid_dir = "../../data/processed/4 Segmenter test/text_segmenter/valid/images"
labels_text_segmenter_valid_dir = "../../data/processed/4 Segmenter test/text_segmenter/valid/labels"

images_text_segmenter_test_dir = "../../data/processed/4 Segmenter test/text_segmenter/test/images"
labels_text_segmenter_test_dir = "../../data/processed/4 Segmenter test/text_segmenter/test/labels"

images_text_recognizer_train_dir = "../../data/processed/4 Segmenter test/text_recognizer/train"

images_text_recognizer_valid_dir = "../../data/processed/4 Segmenter test/text_recognizer/valid"

images_text_recognizer_test_dir = "../../data/processed/4 Segmenter test/text_recognizer/test"

csv_root_path = "../../data/processed/4 Segmenter test"

# Создание каталогов для train, valid, test и images
os.makedirs(images_text_segmenter_train_dir, exist_ok=True)
os.makedirs(labels_text_segmenter_train_dir, exist_ok=True)

os.makedirs(images_text_segmenter_valid_dir, exist_ok=True)
os.makedirs(labels_text_segmenter_valid_dir, exist_ok=True)

os.makedirs(images_text_segmenter_test_dir, exist_ok=True)
os.makedirs(labels_text_segmenter_test_dir, exist_ok=True)

os.makedirs(images_text_recognizer_train_dir, exist_ok=True)
os.makedirs(images_text_recognizer_valid_dir, exist_ok=True)
os.makedirs(images_text_recognizer_test_dir , exist_ok=True)


# Указываем директории и файл с аннотациями
image_dir = Path("../../data/raw/Распознавание текстов/Победоносцев/images")
annotation_file = Path('../../data/raw/Распознавание текстов/Победоносцев/project-14-at-2024-03-27-22-12-07be0e2d.json')

# Проверяем, существует ли файл аннотаций
if not annotation_file.exists():
    raise FileNotFoundError(f"Annotation file not found: {annotation_file}")

# Читаем файл с аннотациями
segment_annotations = pd.read_json(annotation_file)

# Инициализируем список для хранения путей к изображениям
segment_images = []

# Перебор всех файлов изображений в image_dir
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = Path(root) / file
            segment_images.append(image_path)

# Проверка на пустой список изображений
if not segment_images:
    raise ValueError(f"No valid image files found in directory: {image_dir}")

# Рандомизация списка изображений перед разделением
random.shuffle(segment_images)

# Разделяем на тренировочный, тестовый и валидационный наборы
train_images, test_images = train_test_split(segment_images, test_size=0.2, random_state=42)
train_images, valid_images = train_test_split(train_images, test_size=0.2, random_state=42)

# Дополнительная проверка на синхронизацию аннотаций и изображений
train_images = [img for img in train_images if img.name in segment_annotations['file_upload'].values]
valid_images = [img for img in valid_images if img.name in segment_annotations['file_upload'].values]
test_images = [img for img in test_images if img.name in segment_annotations['file_upload'].values]

# Выводим количество изображений в каждом наборе
print(f"Train set size: {len(train_images)}")
print(f"Validation set size: {len(valid_images)}")
print(f"Test set size: {len(test_images)}")


In [ ]:
segment_annotations['file_upload'].nunique()

In [ ]:
print(
    f"Размер обучающей выборки: {len(train_images):>6} \n"
    f"Размер валидационной выборки: {len(valid_images)} \n"
    f"Размер тестовой выборки: {len(test_images):>7}" 
)

In [ ]:
segment_annotations.head()

In [ ]:
# Получаем список ID с лейблом "Перечеркнутый текст", чтобы не включать в обучение т.к. слишком мало данных (всего 5 строк)

id_to_skip = []

for row_id, annotations in segment_annotations[["annotations"]].iterrows():
    for annotations_file in annotations.values[0]:
        for annotation in annotations_file["result"]:
            annotation_id = annotation["id"]
            if annotation["type"] == "labels":
                if annotation["value"]["labels"][0] == "Перечеркнутый текст":
                    id_to_skip.append(annotation_id)

In [ ]:
# Получаем список ID с лейблом "Перечеркнутый текст", чтобы не включать в обучение т.к. слишком мало данных (всего 5 строк)
id_to_skip = []

# Функция для обработки каждой строки DataFrame
def check_annotations(row):
    for annotations_file in row["annotations"]:
        if "result" in annotations_file:
            for annotation in annotations_file["result"]:
                if annotation.get("type") == "labels":
                    labels = annotation.get("value", {}).get("labels", [])
                    if labels and labels[0] == "Перечеркнутый текст":
                        id_to_skip.append(annotation["id"])

# Применяем функцию ко всему DataFrame
segment_annotations.apply(check_annotations, axis=1)
id_to_skip

In [ ]:
# Формируем датасет в pandas

text = []
points = []
orig_points = []
images = []

for row_id, (annotations, image_name) in segment_annotations[["annotations", "file_upload"]].iterrows():
    for annotations_file in annotations:
        for annotation in annotations_file["result"]:
            # Пропускаем разметку с перечеркнутым текстом
            if annotation["id"] in id_to_skip:
                continue
            if annotation["type"] == "textarea":
                # сохраняем текст разметки
                text.append(annotation["value"]["text"][0])
                # нормируем и сохраняем координаты маски
                width = annotation["original_width"]
                height = annotation["original_height"]
                norm_points = []
                yolo_points = []
                for non_norm_points in annotation["value"]["points"]:
                    norm_points.append([
                        (non_norm_points[1] * height) / 100,
                        (non_norm_points[0] * width) / 100
                    ])
                    yolo_points.append([
                        non_norm_points[0] / 100,
                        non_norm_points[1] / 100
                    ])

                points.append(norm_points)
                orig_points.append(yolo_points)
                images.append(image_name)


In [ ]:
# Функция для обработки одной аннотации
def process_annotation(annotation, image_name, id_to_skip, text, points, orig_points, images):
    # Пропускаем аннотацию, если ее ID находится в списке id_to_skip
    if annotation.get("id") in id_to_skip:
        return

    if annotation.get("type") == "textarea":
        # Проверяем, что текстовое поле не пустое
        if annotation.get("value", {}).get("text"):
            text.append(annotation["value"]["text"][0])
        else:
            return  # Если текст пустой, пропускаем

        # Получаем размеры изображения и проверяем их наличие
        width = annotation.get("original_width")
        height = annotation.get("original_height")
        if not width or not height:
            return  # Пропускаем аннотацию, если размеры не указаны

        # Масштабируем и сохраняем координаты маски
        norm_points = []
        yolo_points = []
        for non_norm_points in annotation.get("value", {}).get("points", []):
            norm_points.append([
                (non_norm_points[1] * height) / 100,
                (non_norm_points[0] * width) / 100
            ])
            yolo_points.append([
                non_norm_points[0] / 100,
                non_norm_points[1] / 100
            ])

        points.append(norm_points)
        orig_points.append(yolo_points)
        images.append(image_name)

# Формируем датасет
text = []
points = []
orig_points = []
images = []

# Обработка каждой строки DataFrame
for row_id, (annotations, image_name) in segment_annotations[["annotations", "file_upload"]].iterrows():
    for annotations_file in annotations:
        for annotation in annotations_file.get("result", []):
            process_annotation(annotation, image_name, id_to_skip, text, points, orig_points, images)


In [ ]:
data = pd.DataFrame(data={
    "text": text,
    "points": points,
    "orig_points": orig_points,
    "image": images
})

In [ ]:
yolo_data = data.groupby("image").agg({"points": list, "orig_points": list, "text": list}).reset_index()

In [ ]:
yolo_data.head()

In [ ]:
import cv2
import numpy as np


def crop_and_save_polygon(image_path, out_path, polygon_coords, bbox=None):
    # Загружаем изображение
    image = cv2.imread(image_path)
    
    # Определяем средний цвет изображения для заливки
    mean_color = np.array(cv2.mean(image)).astype(int)[:3]
    
    # Возвращаем на место перепутанные местами координаты yx->xy
    res = []
    for y, x in polygon_coords:
        res.append([x, y])
    
    # Преобразуем координаты полигона в формат, понятный OpenCV
    pts = np.array(res, np.int32)
    pts = pts.reshape((-1,1,2))
    
    # Создаем маску, которая соответствует области полигона
    mask = np.zeros(image.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], 0, 255, -1)
    
    # Вырезаем область изображения, соответствующую маске
    result = cv2.bitwise_and(image, image, mask=mask)
    
    # Заливаем чёрные области тем цветом, который был подан на вход функции
    result[mask==0] = mean_color
    
    # Если задан bbox, обрежем изображение по этим координатам
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        result = result[y1:y2, x1:x2]
    
    cv2.imwrite(out_path, result)
    

def convert_mask_to_xy_bbox(mask):
    # Конвертируем полигон в bbox
    bounding_boxes = sv.polygon_to_xyxy(mask).astype(int)
    # Возвращаем на место перепутанные местами координаты yx->xy
    res_bbox = [bounding_boxes[1], bounding_boxes[0], bounding_boxes[3], bounding_boxes[2]]
    return res_bbox

In [ ]:
import cv2
import numpy as np
# Предполагается, что модуль `sv` импортирован где-то выше
# import sv

def crop_and_save_polygon(image_path, out_path, polygon_coords, bbox=None):
    # Загружаем изображение
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image at {image_path}")
    
    # Определяем средний цвет изображения для заливки
    mean_color = np.array(cv2.mean(image)).astype(int)[:3]
    
    # Преобразуем координаты полигона из [y, x] в [x, y]
    res = [[x, y] for y, x in polygon_coords]
    
    # Преобразуем координаты полигона в формат, понятный OpenCV
    pts = np.array(res, np.int32)
    pts = pts.reshape((-1,1,2))
    
    # Создаем маску, которая соответствует области полигона
    mask = np.zeros(image.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], 0, 255, -1)
    
    # Вырезаем область изображения, соответствующую маске
    result = cv2.bitwise_and(image, image, mask=mask)
    
    # Заливаем чёрные области средним цветом изображения
    result[mask==0] = mean_color
    
    # Если задан bbox, обрезаем изображение по этим координатам
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        x1, y1 = max(x1, 0), max(y1, 0)
        x2, y2 = min(x2, image.shape[1]), min(y2, image.shape[0])
        result = result[y1:y2, x1:x2]
    
    cv2.imwrite(out_path, result)

def convert_mask_to_xy_bbox(mask):
    # Конвертируем полигон в bbox
    bounding_boxes = sv.polygon_to_xyxy(mask).astype(int)
    # Преобразуем из [y1, x1, y2, x2] в [x1, y1, x2, y2]
    res_bbox = [bounding_boxes[1], bounding_boxes[0], bounding_boxes[3], bounding_boxes[2]]
    return res_bbox


In [ ]:
for data_images, (images_data_dir, labels_data_dir), recognizer_images_path, csv_name in tqdm(zip(
        [train_images, valid_images, test_images], 
        [
            [images_text_segmenter_train_dir, labels_text_segmenter_train_dir],
            [images_text_segmenter_valid_dir, labels_text_segmenter_valid_dir],
            [images_text_segmenter_test_dir, labels_text_segmenter_test_dir]
        ],
        [images_text_recognizer_train_dir, images_text_recognizer_valid_dir, images_text_recognizer_test_dir],
        ["train.csv", "valid.csv", "test.csv"]
), total=3):  
    
    objects = []
    
    for image in data_images:
        image_path = Path(image)
        image_name = image_path.name
        image_suffix = image_path.suffix
        
        # # # # # # # # # # # # # # # # # # # # # # # # # # # 
        # # # # # # #   Датасет для сегментации   # # # # # #
        # # # # # # # # # # # # # # # # # # # # # # # # # # #
        
        skip_ocr = False
        
        # копируем изображение
        shutil.copy(image, images_data_dir)
        image_label_path = Path(labels_data_dir) / Path(image_name.replace(image_suffix, ".txt"))
        # создаём файл с разметкой этого изображения
        with open(image_label_path, "w") as file:
                for line in yolo_data[yolo_data["image"] == image_name]["orig_points"].values[0]:
                    if np.array(line).shape[1] > 2:
                        line = line.tolist()
                    file.write("0 " + ' '.join(f"{item[0]} {item[1]}" for item in line) + "\n")

                
        # # # # # # # # # # # # # # # # # # # # # # # # # # # #  
        # # # # # # #   Датасет для распознавания   # # # # # #
        # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
        
        if not skip_ocr:
            ocr_masks, texts = [*yolo_data[yolo_data["image"] == image_name][["points", "text"]].values[0].tolist()]
            for num_mask, [mask, text] in enumerate(zip(ocr_masks, texts)):
                bbox = convert_mask_to_xy_bbox(mask)
                filename = Path(f"{num_mask}_{Path(image).name}")
                out_path = Path(recognizer_images_path) / filename
                crop_and_save_polygon(image, str(out_path), mask, bbox)
                
                objects.append({"file_name": filename, "text": text})
    
    ocr_data = pd.DataFrame(objects)
    ocr_data.to_csv(f"{csv_root_path}/{csv_name}", index=False)

In [ ]:
import shutil
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

def process_image_data(yolo_data, train_images, valid_images, test_images, 
                       segment_dirs, recognizer_dirs, csv_names, csv_root_path):
    
    for data_images, (images_data_dir, labels_data_dir), recognizer_images_path, csv_name in tqdm(
            zip([train_images, valid_images, test_images], 
                segment_dirs, recognizer_dirs, csv_names), total=3):

        objects = []

        for image in data_images:
            image_path = Path(image)
            image_name = image_path.name
            image_suffix = image_path.suffix

            # Загружаем соответствующие аннотации
            image_annotations = yolo_data[yolo_data["image"] == image_name]
            if image_annotations.empty:
                print(f"Warning: No annotations found for image {image_name}. Skipping.")
                continue

            # Копируем изображение в директорию сегментации
            shutil.copy(image, images_data_dir)
            image_label_path = Path(labels_data_dir) / image_name.replace(image_suffix, ".txt")

            # Создаем файл с разметкой для сегментации
            with open(image_label_path, "w") as file:
                for line in image_annotations["orig_points"].values[0]:
                    if len(line[0]) > 2:
                        line = line.tolist()
                    file.write("0 " + ' '.join(f"{item[0]} {item[1]}" for item in line) + "\n")

            # Обрабатываем данные для OCR
            if not skip_ocr:
                ocr_masks, texts = image_annotations[["points", "text"]].values[0].tolist()
                for num_mask, (mask, text) in enumerate(zip(ocr_masks, texts)):
                    bbox = convert_mask_to_xy_bbox(mask)
                    filename = Path(f"{num_mask}_{image_name}")
                    out_path = recognizer_images_path / filename
                    crop_and_save_polygon(image, str(out_path), mask, bbox)

                    objects.append({"file_name": filename, "text": text})

        # Сохранение данных OCR в CSV-файл
        if objects:
            ocr_data = pd.DataFrame(objects)
            ocr_data.to_csv(Path(csv_root_path) / csv_name, index=False)
        else:
            print(f"No OCR data was generated for {csv_name}.")

# Пример вызова функции
process_image_data(yolo_data, train_images, valid_images, test_images,
                   [
                       [images_text_segmenter_train_dir, labels_text_segmenter_train_dir],
                       [images_text_segmenter_valid_dir, labels_text_segmenter_valid_dir],
                       [images_text_segmenter_test_dir, labels_text_segmenter_test_dir]
                   ],
                   [images_text_recognizer_train_dir, images_text_recognizer_valid_dir, images_text_recognizer_test_dir],
                   ["train.csv", "valid.csv", "test.csv"],
                   csv_root_path)


In [ ]:
yolo_data

In [ ]:
skip_ocr

In [ ]:
ocr_data